<a href="https://colab.research.google.com/github/volleyfreak/SETI/blob/main/Simple_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms
import pandas as pd
import numpy as np

In [ ]:
transform = transforms.Compose([#transforms.RandomRotation(30),  # data augmentations are great
                                       #transforms.RandomResizedCrop(224),  # but not in this case of map tiles
                                       #transforms.RandomHorizontalFlip(),
                                       transforms.Resize(50),
                                       #transforms.ToTensor(),
                                       #transforms.Normalize([0.485, 0.456, 0.406], # PyTorch recommends these but in this
                                       #                     [0.229, 0.224, 0.225]) # case I didn't get good results
                                       ])
batch_size = 4
def npy_loader(path):
    sample = torch.from_numpy(np.load(path))
    return sample


dataset = datasets.DatasetFolder(
    root='./drive/MyDrive/Colab Notebooks/',
    loader=npy_loader,
    extensions=('.npy'),
    transform=transform
)

In [ ]:



valid_size = .2
num_train = len(dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))
np.random.shuffle(indices)

from torch.utils.data.sampler import SubsetRandomSampler
train_idx, test_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)
trainloader = torch.utils.data.DataLoader(dataset, sampler=train_sampler, batch_size=1)
testloader = torch.utils.data.DataLoader(dataset, sampler=test_sampler, batch_size=1)

# testset = torchvision.datasets.CIFAR10(root='./data', train=False,
#                                        download=True, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
#                                          shuffle=False, num_workers=2)
#
classes = ('alien', 'no_alien')
import matplotlib.pyplot as plt
import numpy as npy_loader
print(testloader)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(1440, 200)
        self.fc2 = nn.Linear(200, 30)
        self.fc3 = nn.Linear(30, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#model.cuda()
# optimizer.to('gpu')

# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()
#print(len(labels[0]))
print(labels[0])
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(1)))

tensor(0)
GroundTruth:  alien


In [ ]:
print(images.shape)

torch.Size([1, 6, 53, 50])


In [ ]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cuda"
print(device)
net.to(device)
net.cuda()
torch.cuda.empty_cache()
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        data[0] = np.take(data[0][0], [0,2,4], 0)
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.float()
        inputs = inputs.unsqueeze(0)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
print('Finished Training')

cuda
[1,  2000] loss: 0.695
[1,  4000] loss: 0.695
[1,  6000] loss: 0.694
[1,  8000] loss: 0.695
[2,  2000] loss: 0.695
[2,  4000] loss: 0.693
[2,  6000] loss: 0.695
[2,  8000] loss: 0.693
[3,  2000] loss: 0.694
[3,  4000] loss: 0.694
[3,  6000] loss: 0.693
[3,  8000] loss: 0.693
[4,  2000] loss: 0.693
[4,  4000] loss: 0.694
[4,  6000] loss: 0.694
[4,  8000] loss: 0.694
[5,  2000] loss: 0.694
[5,  4000] loss: 0.694
[5,  6000] loss: 0.693
[5,  8000] loss: 0.695
Finished Training


In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        data[0] = np.take(data[0][0], [0,2,4], 0)
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.float()
        inputs = inputs.unsqueeze(0)
        # calculate outputs by running images through the network
        outputs = net(inputs)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {len(test_idx)} test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 2000 test images: 49 %
